In [1]:
import pandas as pd

# Read CSV, skipping the first row (header)
df = pd.read_csv('dataset/train.csv', skiprows=1, header=None)

# Access the 3rd column (index 2)
captions = df[2]

# Count unique values
unique_count = captions.nunique()

print(f"Number of unique values in the 3rd column: {unique_count}")

Number of unique values in the 3rd column: 39


In [3]:
import pandas as pd

# Read CSV, skipping the first row (header)
df = pd.read_csv('dataset/val.csv', skiprows=1, header=None)

# Access the 3rd column (index 2)
captions = df[2]

# Count unique values
unique_count = captions.nunique()

print(f"Number of unique values in the 3rd column: {unique_count}")

Number of unique values in the 3rd column: 33


In [8]:
import pandas as pd

# Load CSVs, skipping the first row (assumed header)
train_df = pd.read_csv('dataset/train.csv', skiprows=1, header=None)
val_df = pd.read_csv('dataset/val.csv', skiprows=1, header=None)

# Extract captions (3rd column)
train_captions = set(train_df[2])
val_captions = set(val_df[2])

# Find captions in val that are not in train
new_captions_in_val = val_captions - train_captions

# Output
# print(f"Total new captions in val.csv not present in train.csv: {len(new_captions_in_val)}\n")
# print("These are the new captions:")
# for caption in new_captions_in_val:
#     print(caption)

all_unique_captions = sorted(train_captions.union(val_captions))

# Output
print(f"Total unique captions in both train.csv and val.csv: {len(all_unique_captions)}\n")
print("List of all unique captions:")
for caption in all_unique_captions:
    print(caption)

Total unique captions in both train.csv and val.csv: 42

List of all unique captions:
Car image with broken lamp, scratch.
Car image with broken lamp.
Car image with crack, broken lamp, shattered glass.
Car image with crack, broken lamp.
Car image with crack, scratch, broken lamp, shattered glass.
Car image with crack, scratch, broken lamp.
Car image with crack, scratch.
Car image with crack, shattered glass.
Car image with crack.
Car image with dent, broken lamp, flat tire, crack, scratch.
Car image with dent, broken lamp, flat tire.
Car image with dent, broken lamp, scratch, flat tire.
Car image with dent, broken lamp, scratch.
Car image with dent, broken lamp, shattered glass, crack, scratch.
Car image with dent, broken lamp, shattered glass.
Car image with dent, broken lamp.
Car image with dent, crack, broken lamp, shattered glass.
Car image with dent, crack, broken lamp.
Car image with dent, crack, flat tire, broken lamp.
Car image with dent, crack, flat tire.
Car image with dent,

In [9]:
import base64
import requests

# === CONFIG ===
image_path = "dataset/val/val/000103.jpg"  # Replace with your actual image path
api_url = "https://api.core42.ai/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"
api_token = "2c53e996ac264f8497cfbf6428d777e0"  # Replace with your actual token

# === Read and encode image ===
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

# === Prepare prompt ===
prompt = """I have uploaded an image. 
Based on damage either one or many from the list, please compose output.
List: [crack, scratch, broken lamp, shattered glass, dent]
Output: Car image with ___

Please only respond back with output."""

# === Construct payload ===
payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }
    ],
    "max_tokens": 300
}

# === Send request ===
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_token}"
}

response = requests.post(api_url, headers=headers, json=payload)

print(response.json()["choices"][0]["message"]["content"])

Car image with shattered glass and dent.


In [15]:
# If this works, I'll say BINGOO

import pandas as pd
import base64
import requests
import os
import time

api_url = "https://api.core42.ai/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"
api_token = "2c53e996ac264f8497cfbf6428d777e0"
input_csv = "dataset/test.csv"
image_base_path = "dataset/test/test"
output_csv = "submission3.csv"

prompt = """I have uploaded an image. 
Based on damage either one or many from the list, please compose output.
List: [crack, scratch, broken lamp, shattered glass, dent, flat tire]
Output: Car image with ___

Please only respond back with output."""

df = pd.read_csv(input_csv)
results = []

for idx, row in df.iterrows():
    image_id = row['id']
    file_path = os.path.join(image_base_path, row['file_name'])

    if not os.path.exists(file_path):
        print(f"[{idx+1}/{len(df)}] Image not found: {file_path}")
        results.append({"id": image_id, "prediction": "Image not found"})
        continue

    with open(file_path, "rb") as f:
        base64_image = base64.b64encode(f.read()).decode("utf-8")

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_token}"
    }

    try:
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        caption = response.json()["choices"][0]["message"]["content"]
        caption = caption.replace("Output:", "").strip()
    except Exception as e:
        caption = f"Error: {e}"
    
    print(f"[{idx+1}/{len(df)}] Processed ID {image_id} — {caption[:60]}...")
    results.append({"id": image_id, "prediction": caption})
    time.sleep(1) 

submission_df = pd.DataFrame(results)
submission_df.to_csv(output_csv, index=False)
print(f"\nSubmission file:: {output_csv}")

[1/374] Processed ID 12 — Car image with flat tire....
[2/374] Processed ID 15 — Car image with scratch....
[3/374] Processed ID 23 — Car image with dent and scratch....
[4/374] Processed ID 33 — Car image with dent....
[5/374] Processed ID 40 — Car image with crack, scratch, and dent...
[6/374] Processed ID 42 — Car image with shattered glass, broken lamp, dent....
[7/374] Processed ID 44 — Car image with crack, scratch, and dent....
[8/374] Processed ID 57 — Car image with flat tire....
[9/374] Processed ID 82 — Car image with scratch and dent....
[10/374] Processed ID 88 — Car image with dent....
[11/374] Processed ID 90 — Car image with shattered glass....
[12/374] Processed ID 106 — Car image with broken lamp....
[13/374] Processed ID 128 — Car image with shattered glass and dent....
[14/374] Processed ID 152 — Car image with broken lamp, shattered glass, dent....
[15/374] Processed ID 165 — Car image with shattered glass....
[16/374] Processed ID 175 — Car image with shattered gl

In [14]:
import pandas as pd
df = pd.read_csv("submission2.csv")
invalid_rows = df[~df['prediction'].str.startswith("Car image with", na=False)]
print(f"{len(invalid_rows)} rows do NOT start with 'Car image with'")
print(invalid_rows.head())

0 rows do NOT start with 'Car image with'
Empty DataFrame
Columns: [id, prediction]
Index: []
